In [1]:
import os
import math
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

<strong>There are 2 datasets:</strong><br/>
<ol>
<li>Training data set,</li>
<li>Test data set.</li>
</ol>
Each of these data sets are .csv files. <br/>
Each dataset has 785 columns with a variable number of rows.
We shall use the pandas package to read these files. The results are numpy arrays.
The first column contains the label, the remaining columns contain the pixel data.

In [2]:
root_dir = "C:\\Apps\\Projects\\ml\\data\\mnist\\"
training_filename = root_dir + "mnist_train.csv"
# Use pandas to read csv file
tmp=pd.read_csv(training_filename, sep=',', header=None)
# Use df.loc to access a group of rows and columns
# Use .values to return the numpy representation.

# Read the training data set
x_train, y_train=tmp.loc[:,1:tmp.shape[1]].values, tmp.loc[:,0].values

# Read the test data set
test_filename = root_dir + "mnist_test.csv"
tmp=pd.read_csv(test_filename, sep=',', header=None)
x_test, y_test=tmp.loc[:,1:tmp.shape[1]].values, tmp.loc[:,0].values
tmp=[]

In [3]:
logs_path = 'logging/' # logging path
batch_size = 100 # batch size while performing training 
learning_rate = 0.003 # Learning rate 
training_epochs = 30 # training epoch
display_epoch = 1

In [4]:
# x = tf.placeholder(dtype=tf.int32, shape=[], name="foo1")       shape takes a single scalar value directly.
# y = tf.placeholder(dtype=tf.int32, shape=[None], name="foo2")   shape takes a 1-dim array
# z = tf.placeholder(dtype=tf.int32, shape=None, name="foo3")     shape can take in any value 

In [5]:
# tf.placeholder(dtype, shape=None, name=None)
#
X = tf.placeholder(tf.float32, shape=(None, 784))
# Variable number of 28 x 28 x 1 tensors. -1 indicates the dimension is unknown.
x_image = tf.reshape(X, [-1, 28, 28, 1])
Y_  = tf.placeholder(tf.float32, shape=(None, 10))
lr = tf.placeholder(tf.float32)

In [6]:
# A variable maintains state in the graph across calls to run()
# Create a variable.
# w = tf.Variable(<initial-value>, name=<optional-name>, shape=<optional-shape>)

# Truncated normal Outputs random values from a truncated normal distribution.
#
# tf.random.truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.dtypes.float32, seed=None, name=None)


In [7]:
def wt_var(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

In [8]:
# tf.constant creates a constant tensor.
# tf.constant(value, dtype=None, shape=None, name='Const', verify_shape=False)

In [9]:
def bias_var(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

In [10]:
# Computes a 2-D convolution given 4-D input and filter tensors.
# tf.nn.conv2d(input, filter, strides, padding,
#     use_cudnn_on_gpu=True, data_format='NHWC', dilations=[1, 1, 1, 1], name=None)

# input = A Tensor
# filter = Tensor [filter_height, filter_width, in_channels, out_channels]
# strides: A list of ints. 1-D tensor of length 4. 
#          The stride of the sliding window for each dimension of input
#          Usually has the form strides = [1, stride, stride, 1].
# padding: A string from: "SAME", "VALID". The type of padding algorithm to use.

# Given an input tensor of shape [batch, in_height, in_width, in_channels]
# and a filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
# this op 
# a) Flattens the filter to a 2-D matrix with shape [filter_height * filter_width * in_channels, output_channels].
# b) Extracts image patches from the input tensor to form a virtual tensor of shape 
#   [batch, out_height, out_width, filter_height * filter_width * in_channels]
# c) For each patch, right-multiplies the filter matrix and the image patch vector.



In [11]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

In [12]:
# tf.nn.max_pool(value, ksize, strides, padding, data_format='NHWC', name=None)
# 
# value: A 4-D Tensor of the format specified by data_format.
# ksize: A list or tuple of 4 ints. The size of the window for each dimension of the input tensor.
# strides: A list or tuple of 4 ints. The stride of the sliding window for each dimension of the input tensor.
# padding: A string, either 'VALID' or 'SAME'. The padding algorithm.
# data_format: A string. 'NHWC', 'NCHW' and 'NCHW_VECT_C' are supported.
# name: Optional name for the operation.

In [13]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [14]:
# tf.nn.relu(features, name=None)
# 
# features: A Tensor. 
# name: A name for the operation (optional).

In [15]:
W_conv1 = wt_var([3, 3, 1, 32]) # [filter_height, filter_width, in_channels, out_channels] = [28, 28, 1, 32]
b_conv1 = bias_var([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [16]:
W_conv2 = wt_var([3, 3, 32, 64]) # [filter_height, filter_width, in_channels, out_channels] = [14, 14, 1, 32]
b_conv2 = bias_var([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2) # Output = [7, 7, 1, 64]

In [17]:
# Fully connected layer
W_fc1 = wt_var([7 * 7 * 64, 1024]) # [7 x 7 x 64, 1024]
b_fc1 = bias_var([1024])

h_pool2_vec = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_vec, W_fc1) + b_fc1)

In [18]:
keep_prob = tf.placeholder("float32")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [19]:
W_fc2 = wt_var([1024, 10])  # Softmax layer
b_fc2 = bias_var([10])

#y_hat = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
Ylogits = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
y_hat = tf.nn.softmax(Ylogits)

In [20]:
#cross_entropy = tf.reduce_sum(Y_ * tf.log(y_hat))
#cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=Ylogits, labels=Y_)
cost_op = tf.reduce_mean(cross_entropy)*100
#cost_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_))

In [21]:
correct_prediction = tf.equal(tf.argmax(y_hat, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=Ylogits, labels=Y_)
#cost_op = tf.reduce_mean(cross_entropy)*100

In [22]:
# Optimization op (backprop)
#train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
train_op = tf.train.AdamOptimizer(learning_rate).minimize(cost_op)

In [23]:
init_op = tf.global_variables_initializer()

In [24]:
print('Enter cell ...')
with tf.Session() as sess:
    # Run the initializer
    print('Training model ...')
    sess.run(init_op)
    #avg_cost = 0.
    # op to write logs to TensorBoard
    writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    yM_train = np.zeros(shape=(len(y_train), 10), dtype=float)
    for i in range(len(y_train)):
        yM_train[i, y_train[i]] = 1
    yM_test = np.zeros(shape=(len(y_test), 10), dtype=float)
    for i in range(len(y_test)):
        yM_test[i, y_test[i]] = 1
    batch_count = int(len(y_train)/batch_size)
    #print("Batch count: ", batch_count)
    for epoch in range(training_epochs):
        for i in range(batch_count):
            start_row = i*batch_size
            end_row_plus_one = (i+1)*batch_size
            # Change this
            max_learning_rate = 0.003
            min_learning_rate = 0.0001
            decay_speed = 2000 
            batch_x, batch_y = x_train[start_row:end_row_plus_one, :], yM_train[start_row:end_row_plus_one,:]

            learning_rate = min_learning_rate+\
                            (max_learning_rate - min_learning_rate)\
                            * math.exp(-i/decay_speed)
            _ = sess.run([train_op], feed_dict={X: batch_x, Y_: batch_y, lr: learning_rate, keep_prob:0.5})
            #avg_cost += c / batch_count
            #writer.add_summary(epoch * batch_count + i)
        if (epoch+1) % display_epoch == 0:
            print("Epoch:", '%2d' % (epoch+1)) #, "cost=", "{:.2f}".format(avg_cost))   
        #print("Epoch: ", epoch)
    print("Optimization Finished!")
           
    print("Accuracy: ", accuracy.eval(feed_dict={X: x_test, Y_: yM_test, keep_prob:1.0}))

Enter cell ...
Training model ...
Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9
Epoch: 10
Epoch: 11
Epoch: 12
Epoch: 13
Epoch: 14
Epoch: 15
Epoch: 16
Epoch: 17
Epoch: 18
Epoch: 19
Epoch: 20
Epoch: 21
Epoch: 22
Epoch: 23
Epoch: 24
Epoch: 25
Epoch: 26
Epoch: 27
Epoch: 28
Epoch: 29
Epoch: 30
Optimization Finished!
Accuracy:  0.9818


In [25]:
index=15
print(yM_test[index,:])
print(y_test[index])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
5
